<a href="https://colab.research.google.com/github/MineManiac/superComputacao-2024.2/blob/main/EntregaAula11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile exercicio.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/functional.h>    // Inclui operadores e functors do Thrust
#include <iostream>
#include <fstream>   // Para manipulação de arquivos
#include <sstream>   // Para manipulação de strings

// Functor para calcular a diferença entre dois valores
struct diff_functor {
    __host__ __device__
    double operator()(const double& a, const double& b) const {
        return a - b;
    }
};

// Functor para calcular o quadrado da diferença em relação à média
struct variance_functor {
    double mean;

    variance_functor(double _mean) : mean(_mean) {}

    __host__ __device__
    double operator()(const double& x) const {
        double diff = x - mean;
        return diff * diff;
    }
};

int main() {
    const int n = 3520;  // Aproximadamente 3520 registros no arquivo (10 anos de dados)
    thrust::host_vector<double> host_apple(n);
    thrust::host_vector<double> host_microsoft(n);

    // Abrir o arquivo stocks2.txt
    std::ifstream file("stocks2.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks2.txt!" << std::endl;
        return 1;
    }

    // Ler os dados do arquivo linha por linha
    std::string line;
    int i = 0;
    while (std::getline(file, line) && i < n) {
        std::stringstream ss(line);
        std::string apple_price, microsoft_price;

        // Pega os dois valores separados por vírgula
        std::getline(ss, apple_price, ',');
        std::getline(ss, microsoft_price, ',');

        // Armazena os preços no vetor host
        host_apple[i] = std::stod(apple_price);  // Converte para double
        host_microsoft[i] = std::stod(microsoft_price);
        i++;
    }
    file.close();

    // Copiar os dados do host para a GPU
    thrust::device_vector<double> dev_apple = host_apple;
    thrust::device_vector<double> dev_microsoft = host_microsoft;
    thrust::device_vector<double> dev_diff(n);  // Vetor para armazenar as diferenças
    thrust::device_vector<double> dev_variance(n);  // Vetor para armazenar os quadrados das diferenças

    // Calcular a diferença ponto a ponto entre os dois vetores
    thrust::transform(dev_apple.begin(), dev_apple.end(), dev_microsoft.begin(), dev_diff.begin(), diff_functor());

    // Calcular a soma das diferenças
    double sum_diff = thrust::reduce(dev_diff.begin(), dev_diff.end(), 0.0, thrust::plus<double>());

    // Calcular a média das diferenças
    double mean_diff = sum_diff / n;

    // Calcular a variância usando thrust::transform e thrust::reduce
    thrust::transform(dev_diff.begin(), dev_diff.end(), dev_variance.begin(), variance_functor(mean_diff));
    double variance = thrust::reduce(dev_variance.begin(), dev_variance.end(), 0.0, thrust::plus<double>());
    variance /= n;

    // Exibir os resultados
    std::cout << "A diferença média entre os preços da Apple e Microsoft é: " << mean_diff << std::endl;
    std::cout << "A variância das diferenças é: " << variance << std::endl;

    return 0;
}



Writing exercicio.cu


In [3]:
!nvcc -arch=sm_70 -std=c++14 exercicio.cu -o exercicio


In [4]:
!./exercicio < stocks2.txt

A diferença média entre os preços da Apple e Microsoft é: -265.232
A variância das diferenças é: 16120.3


In [10]:
%%writefile exercicio2.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include <stdexcept>  // For exception handling

// Functor para calcular a diferença entre o preço atual e o anterior
struct daily_gain_functor {
    __host__ __device__
    double operator()(const double& next, const double& current) const {
        return next - current;
    }
};

int main() {
    thrust::host_vector<double> host_stocks;

    // Abrir o arquivo stocks-google.txt
    std::ifstream file("stocks-google.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt!" << std::endl;
        return 1;
    }

    // Ler os dados do arquivo linha por linha
    std::string line;
    while (std::getline(file, line)) {
        // Verificar se a linha não está vazia e contém dados válidos
        try {
            if (!line.empty()) {
                double price = std::stod(line);  // Converte cada linha para double
                host_stocks.push_back(price);    // Armazena o valor no vetor host
            }
        } catch (const std::invalid_argument& e) {
            std::cerr << "Erro: valor inválido na linha: " << line << std::endl;
            continue;  // Ignorar a linha inválida e continuar
        } catch (const std::out_of_range& e) {
            std::cerr << "Erro: valor fora do intervalo na linha: " << line << std::endl;
            continue;  // Ignorar a linha inválida e continuar
        }
    }
    file.close();

    int n = host_stocks.size();  // Atualizar n com o número real de registros lidos
    if (n < 2) {
        std::cerr << "Erro: Não há dados suficientes para calcular o ganho diário!" << std::endl;
        return 1;
    }

    // Copiar os dados do host para a GPU
    thrust::device_vector<double> dev_stocks = host_stocks;
    thrust::device_vector<double> dev_ganho_diario(n - 1);  // O vetor de diferença terá tamanho n-1

    // Calcular a diferença diária entre o preço de um dia e o anterior
    thrust::transform(dev_stocks.begin(), dev_stocks.end() - 1,    // Vetor atual
                      dev_stocks.begin() + 1,                      // Vetor deslocado
                      dev_ganho_diario.begin(),                    // Armazena a saída
                      daily_gain_functor());                       // Functor que calcula a diferença

    // Exibir as primeiras diferenças
    for (int j = 0; j < std::min(10, n-1); j++) {
        std::cout << "Ganho diário [" << j << "]: " << dev_ganho_diario[j] << std::endl;
    }

    return 0;
}


Writing exercicio2.cu


In [11]:
!nvcc -arch=sm_70 -std=c++14 exercicio2.cu -o exercicio2

In [12]:
!./exercicio2 < stocks-google.txt

Ganho diário [0]: -5.26
Ganho diário [1]: -1.5
Ganho diário [2]: 3.25
Ganho diário [3]: -6.25
Ganho diário [4]: -3.5
Ganho diário [5]: -3.25
Ganho diário [6]: 1.75
Ganho diário [7]: -5
Ganho diário [8]: 2.5
Ganho diário [9]: -5.5


In [14]:
%%writefile exercicio3.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/count.h>
#include <iostream>
#include <fstream>
#include <string>
#include <stdexcept>  // Para manipulação de exceções

// Functor para calcular a diferença entre o preço atual e o anterior
struct daily_gain_functor {
    __host__ __device__
    double operator()(const double& next, const double& current) const {
        return next - current;
    }
};

// Functor para contar os ganhos positivos
struct is_positive {
    __host__ __device__
    bool operator()(const double& x) const {
        return x > 0;
    }
};

int main() {
    thrust::host_vector<double> host_stocks;

    // Abrir o arquivo stocks-google.txt
    std::ifstream file("stocks-google.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt!" << std::endl;
        return 1;
    }

    // Ler os dados do arquivo linha por linha
    std::string line;
    while (std::getline(file, line)) {
        try {
            if (!line.empty()) {
                double price = std::stod(line);  // Converte cada linha para double
                host_stocks.push_back(price);    // Armazena o valor no vetor host
            }
        } catch (const std::invalid_argument& e) {
            std::cerr << "Erro: valor inválido na linha: " << line << std::endl;
            continue;  // Ignorar a linha inválida e continuar
        } catch (const std::out_of_range& e) {
            std::cerr << "Erro: valor fora do intervalo na linha: " << line << std::endl;
            continue;  // Ignorar a linha inválida e continuar
        }
    }
    file.close();

    int n = host_stocks.size();  // Atualizar n com o número real de registros lidos
    if (n < 2) {
        std::cerr << "Erro: Não há dados suficientes para calcular o ganho diário!" << std::endl;
        return 1;
    }

    // Copiar os dados do host para a GPU
    thrust::device_vector<double> dev_stocks = host_stocks;
    thrust::device_vector<double> dev_ganho_diario(n - 1);  // O vetor de diferença terá tamanho n-1

    // Calcular a diferença diária entre o preço de um dia e o anterior
    thrust::transform(dev_stocks.begin(), dev_stocks.end() - 1,    // Vetor atual
                      dev_stocks.begin() + 1,                      // Vetor deslocado
                      dev_ganho_diario.begin(),                    // Armazena a saída
                      daily_gain_functor());                       // Functor que calcula a diferença

    // Contar quantas vezes o preço das ações subiu (ganhos positivos)
    int subidas = thrust::count_if(dev_ganho_diario.begin(), dev_ganho_diario.end(), is_positive());

    // Exibir o resultado
    std::cout << "O preço das ações subiu " << subidas << " vezes." << std::endl;

    return 0;
}


Writing exercicio3.cu


In [15]:
!nvcc -arch=sm_70 -std=c++14 exercicio3.cu -o exercicio3

In [16]:
!./exercicio3 < stocks-google.txt

O preço das ações subiu 55 vezes.


In [17]:
%%writefile exercicio4.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>
#include <string>
#include <stdexcept>  // Para manipulação de exceções

// Functor para calcular a diferença entre o preço atual e o anterior
struct daily_gain_functor {
    __host__ __device__
    double operator()(const double& next, const double& current) const {
        return next - current;
    }
};

// Functor para contar os ganhos positivos
struct is_positive {
    __host__ __device__
    bool operator()(const double& x) const {
        return x > 0;
    }
};

// Functor para substituir valores negativos
struct is_negative {
    __host__ __device__
    bool operator()(const double& x) const {
        return x < 0;
    }
};

int main() {
    thrust::host_vector<double> host_stocks;

    // Abrir o arquivo stocks-google.txt
    std::ifstream file("stocks-google.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt!" << std::endl;
        return 1;
    }

    // Ler os dados do arquivo linha por linha
    std::string line;
    while (std::getline(file, line)) {
        try {
            if (!line.empty()) {
                double price = std::stod(line);  // Converte cada linha para double
                host_stocks.push_back(price);    // Armazena o valor no vetor host
            }
        } catch (const std::invalid_argument& e) {
            std::cerr << "Erro: valor inválido na linha: " << line << std::endl;
            continue;  // Ignorar a linha inválida e continuar
        } catch (const std::out_of_range& e) {
            std::cerr << "Erro: valor fora do intervalo na linha: " << line << std::endl;
            continue;  // Ignorar a linha inválida e continuar
        }
    }
    file.close();

    int n = host_stocks.size();  // Atualizar n com o número real de registros lidos
    if (n < 2) {
        std::cerr << "Erro: Não há dados suficientes para calcular o ganho diário!" << std::endl;
        return 1;
    }

    // Copiar os dados do host para a GPU
    thrust::device_vector<double> dev_stocks = host_stocks;
    thrust::device_vector<double> dev_ganho_diario(n - 1);  // O vetor de diferença terá tamanho n-1

    // Calcular a diferença diária entre o preço de um dia e o anterior
    thrust::transform(dev_stocks.begin(), dev_stocks.end() - 1,    // Vetor atual
                      dev_stocks.begin() + 1,                      // Vetor deslocado
                      dev_ganho_diario.begin(),                    // Armazena a saída
                      daily_gain_functor());                       // Functor que calcula a diferença

    // Contar quantas vezes o preço das ações subiu (ganhos positivos)
    int subidas = thrust::count_if(dev_ganho_diario.begin(), dev_ganho_diario.end(), is_positive());

    // Substituir todos os valores negativos de ganho_diario por zero
    thrust::replace_if(dev_ganho_diario.begin(), dev_ganho_diario.end(), is_negative(), 0.0);

    // Calcular a soma dos ganhos (agora com os negativos substituídos por zero)
    double soma_ganhos = thrust::reduce(dev_ganho_diario.begin(), dev_ganho_diario.end(), 0.0);

    // Calcular a média dos aumentos
    double media_aumentos = (subidas > 0) ? (soma_ganhos / subidas) : 0.0;

    // Exibir o resultado
    std::cout << "O preço das ações subiu " << subidas << " vezes." << std::endl;
    std::cout << "A média dos aumentos foi: " << media_aumentos << std::endl;

    return 0;
}


Writing exercicio4.cu


In [18]:
!nvcc -arch=sm_70 -std=c++14 exercicio4.cu -o exercicio4

In [19]:
!./exercicio4 < stocks-google.txt

O preço das ações subiu 55 vezes.
A média dos aumentos foi: 153.624


Usar GPU para cálculos com Thrust oferece ganhos significativos em desempenho devido à paralelização e operações eficientes, como thrust::transform e thrust::reduce. Iteradores dinâmicos, como thrust::constant_iterator, ajudam a economizar memória ao evitar a alocação de vetores intermediários. No entanto, o tempo de transferência entre CPU e GPU e a memória limitada da GPU podem ser desafios. O uso de GPU é especialmente vantajoso para grandes datasets, mas é crucial balancear o custo da transferência de dados com o ganho de desempenho.